# Анализ базы данные сервиса для чтения книг по подписке

Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

## Введение 

В настоящем исследовании будет изучен датасет сервиса для чтения книг по подписке.

**`Цели исследования:`**
1. Проанализировать базу данных;
2. Ответить на вопросы заказчика;
3. Дать рекомендации заказчику по улчшению сервиса.


**`Задачи:`**
1. Исследовать таблицы — вывести первые строки каждой из таблиц.
2. Посчитать сколько книг вышло после 1 января 2000 года; 
3. Для каждой книги посчитать количество обзоров и среднюю оценку;
4. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключите из анализа брошюры;
5. Определить автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками;
6. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок;
7. Подвести итоги исследования


**`Описание данных:`** 

Таблица **`books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица **`authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица **`publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица **`ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица **`reviews`**

Содержит данные о пользовательских обзорах:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

## Исследование таблицы 

In [1]:
# импортируем библиотеки

import pandas as pd
from sqlalchemy import text, create_engine
from PIL import Image
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Посмтрим первые пять строк в `books`**

In [2]:
books_query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(books_query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Посмтрим первые пять строк в `authors`**

In [3]:
authors_query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(authors_query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Посмтрим первые пять строк в `publishers`**

In [4]:
publishers_query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(publishers_query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Посмтрим первые пять строк в `ratings`**

In [5]:
ratings_query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(ratings_query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Посмтрим первые пять строк в `reviews`**

In [6]:
reviews_query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(reviews_query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Количество книг опубликованных после 1 января 2000 года

In [7]:
books = '''SELECT COUNT(*) 
           FROM books 
           WHERE publication_date >= '2000-01-01';'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(books), con = con)

,count
0,821


821 книга была выпущена после 1 января 2000 года

## Количество обзоров и средняя оценка книг;

In [8]:
count_review = '''SELECT b.book_id AS id, 
                         b.title AS title, 
                         COUNT(DISTINCT r.review_id) AS total_reviews, 
                         ROUND(AVG(ra.rating),2) AS avg_rating
                  FROM books AS b 
                  LEFT JOIN reviews AS r ON b.book_id=r.book_id
                  LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
                  GROUP BY id
                  ORDER BY total_reviews DESC, avg_rating DESC'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(count_review), con = con)

,id,title,total_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Самое большое количество обзоров имеет книга Twilight, однако средний рейтинг у книги небольшой - 3.66 из 5. 

## Издательство, которое выпустило наибольшее число книг 

In [9]:
#Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключите из анализа брошюры
count_books_by_publisher = '''SELECT p.publisher, 
                              COUNT(b.book_id) AS total_books
                              FROM books AS b
                              LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id
                              WHERE b.num_pages >= 50
                              GROUP BY p.publisher
                              ORDER BY total_books DESC
                              LIMIT 1'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(count_books_by_publisher), con = con)

,publisher,total_books
0,Penguin Books,42


**Penguin Books** - издательство, которое выпустило наибольшее число книг толще 50 страниц.

## Автор с самой высокой средней оценкой книг 

In [10]:
#Определим автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками;
top_authors = '''WITH top_books AS 
                 (SELECT a.author AS author, b.book_id, 
                 AVG(r.rating) AS avg_rating
                 FROM books AS b 
                 LEFT JOIN ratings AS r ON b.book_id=r.book_id
                 LEFT JOIN authors AS a ON b.author_id=a.author_id
                 GROUP BY a.author, b.book_id
                 HAVING COUNT(r.rating) >= 50)
                 SELECT author,
                        ROUND(AVG(avg_rating), 2) AS high_book_avg_rating
                 FROM top_books
                 GROUP BY author
                 ORDER BY high_book_avg_rating DESC
                 LIMIT 1'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(top_authors), con = con)   

,author,high_book_avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


**J.K. Rowling/Mary GrandPré** имеет самую высокую среднюю оценку книг - **4.28**.

## Cреднее количество обзоров от пользователей

In [11]:
# Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок;
avg_users_reviews = '''WITH users_by_rating AS 
                       (SELECT username AS users,
                       COUNT(rating_id) AS count_ratings
                       FROM ratings
                       GROUP BY username
                       HAVING COUNT(rating) > 48), 
                       users_by_reviews AS 
                       (SELECT username AS users, 
                        COUNT(review_id) AS count_reviews
                        FROM reviews
                        GROUP BY users)
                        SELECT ROUND(AVG(re.count_reviews), 2) AS avg_count_reviews
                        FROM users_by_reviews AS re
                        JOIN users_by_rating AS ra ON ra.users=re.users
                       '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(avg_users_reviews), con = con)  

,avg_count_reviews
0,24.0


Среднее количество обзоров от пользователей - 24.

## Выводы

1. 821 книга была выпущена после 1 января 2000 года
2. Самое большое количество обзоров имеет книга Twilight, однако средний рейтинг у книги небольшой - 3.66 из 5.
3. Penguin Books - издательство, которое выпустило наибольшее число книг толще 50 страниц (42 книги)
4. J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку книг - 4.28.
5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, - 24.